# Dynamic Time Warping for Non-Anthropomorphic Hand Data

In [1]:
# Set up:
# %matplotlib widget
# %matplotlib inline
# %matplotlib ipympl
# %matplotlib notebook

import pandas as pd
import numpy as np

# Plotting Packages
import matplotlib as mpl
import matplotlib.pyplot as plt

from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

from ipywidgets import *

mpl.rcParams['figure.dpi'] = 150
savefig_options = dict(format="png", dpi=150, bbox_inches="tight")

# import plotly.graph_objects as go
# import plotly.express as px
# import ipywidgets as widget

# Computation packages
from scipy.spatial.distance import euclidean
from scipy.signal import find_peaks
from fastdtw import fastdtw

In [2]:
from nah.alignments import load_raw_csv_data
from nah.utils import norm_data, full_align, full_joint_align,clean_rot_data, load_raw_csv_data, load_npzs, segmentbydemo, sumofsquares

from nah.plot import plot_norm, plot_pos, plot_rot, plot_raw_data, plot_raw_data_subsampled

In [3]:
%matplotlib widget
# %matplotlib inline
# %matplotlib ipympl
# %matplotlib notebook

# robot_name='j2s6s300'
robot_name='Reachy'
gesture_num=11


total_end_eff = np.array([])
total_camera  = np.array([])
total_rh      = np.array([])
total_lh      = np.array([])
total_joint   = np.array([])

singlePID=True
singlePIDval = 12
followup = False

if singlePID:
    PID_begin_range=singlePIDval
    PID_end_range=singlePIDval+1 #Don't forget to +1 to whatever your last PID is
else:
    PID_begin_range=1
    if followup:
        PID_end_range=10 #Don't forget to +1 to whatever your last PID is
    else:
        PID_end_range=17
for PID in range(PID_begin_range,PID_end_range):
    end_eff, camera, rh, lh, joint = load_npzs(robot_name, PID, followup, gesture_num)
    if (PID==PID_begin_range):
        total_end_eff = end_eff
        total_camera = camera
        total_rh = rh
        total_lh = lh
        total_joint = joint
    else:
        total_end_eff = np.vstack((total_end_eff,end_eff))
        total_camera  = np.vstack((total_camera,camera))
        total_rh      = np.vstack((total_rh,rh))
        total_lh      = np.vstack((total_lh,lh))
        total_joint   = np.vstack((total_joint,joint))
# plot_raw_data(end_eff, rh, lh, camera, joint, start_index, end_index)
# plot_raw_data_subsampled(5, total_end_eff, total_camera, total_rh, total_lh, total_joint)

demo_max=5
end_eff, camera, rh, lh, joints = segment_by_demo(total_end_eff, total_camera, total_rh, total_lh, total_joint, demo_max)
for i in range(0,5):
    plot_raw_data_subsampled(5, end_eff[i], camera[i], rh[i], lh[i], joint[i])

plot_raw_data_subsampled(5, total_end_eff, total_camera, total_rh, total_lh, total_joint)

1 (1330, 7) (4,)
2 (1330, 7) (4,)
3 (1330, 7) (4,)
4 (1330, 7) (4,)


IndexError: index 4 is out of bounds for axis 0 with size 4

In [ ]:
from evo.core import metrics, sync
from evo.core.trajectory import PoseTrajectory3D
from scipy.spatial.transform import Rotation
from copy import deepcopy

In [ ]:
def get_evo_trajectory(traj):
    """timestamp, tx, ty, tz, rx, ry, rz"""
    timestamps = traj[:, 0]
    xyz = traj[:, 1:4]

    euler_angles = traj[:, 4:7]
    Rs = Rotation.from_euler('xyz', euler_angles)
    quat_xyzw = Rs.as_quat()
    quat_wxyz = quat_xyzw[:, (3, 0, 1, 2)]
    
    return PoseTrajectory3D(positions_xyz=xyz,
                           orientations_quat_wxyz=quat_wxyz,
                           timestamps=timestamps)

def evo_sync(traj1: PoseTrajectory3D, traj2: PoseTrajectory3D):
    """Synchronize trajectories using Evo's associate_trajectories method"""
    traj1, traj2 = sync.associate_trajectories(traj1, traj2)
    return traj1, traj2
    

def align(traj1, traj2, correct_scale=True):
    """
    Align the first trajectory to the second one.
    Returns the aligned first trajectory.
    """
    traj1_aligned = deepcopy(traj1)
    r, t, s = traj1_aligned.align(traj2, correct_scale=correct_scale)
    print(r, t, s)
    return traj1_aligned
    
def evaluate_ape(traj1: PoseTrajectory3D, traj2: PoseTrajectory3D):
    metric = metrics.APE(metrics.PoseRelation.full_transformation)
    metric.process_data((traj1, traj2))
    
    return metric

In [ ]:
end_eff_traj = get_evo_trajectory(end_eff[0])
right_hand_traj = get_evo_trajectory(rh[0])

In [ ]:
end_eff_traj, right_hand_traj = evo_sync(end_eff_traj, right_hand_traj)

metric = evaluate_ape(end_eff_traj, right_hand_traj)
print(metric.get_all_statistics())

right_hand_traj = align(right_hand_traj, end_eff_traj)

In [ ]:
def convert_evo_to_np(traj: PoseTrajectory3D, shape):
    array = np.empty(shape)
    array[:, 0] = traj.timestamps
    array[:, 1:4] = traj.positions_xyz
    array[:, 4:7] = traj.get_orientations_euler()
    return array

end_eff_aligned = convert_evo_to_np(end_eff_traj, end_eff[0].shape)
print(end_eff_aligned.shape)

rh_aligned = convert_evo_to_np(right_hand_traj, rh[0].shape)
print(rh_aligned.shape)

In [ ]:
plot_raw_data_subsampled(5, end_eff_aligned, camera[0], rh_aligned, lh[0], joint[0])

In [ ]:
metric = evaluate_ape(end_eff_traj, right_hand_traj)
metric.get_all_statistics()

In [ ]:
def get_evo_metrics(traj1, traj2):
    traj1_evo = get_evo_trajectory(traj1)
    traj2_evo = get_evo_trajectory(traj2)   
    
    traj1_evo, traj2_evo = evo_sync(traj1_evo, traj2_evo)
    
    metric = evaluate_ape(traj1_evo, traj2_evo)
    return(metric.get_all_statistics())

def get_aligned_evo_metrics(traj1, traj2):
    traj1_evo = get_evo_trajectory(traj1)
    traj2_evo = get_evo_trajectory(traj2)   
    
    traj1_evo, traj2_evo = evo_sync(traj1_evo, traj2_evo)    
    # metric = evaluate_ape(traj1_evo, traj2_evo)
    
    traj2_evo = align(traj2_evo, traj1_evo)

    traj1_aligned = convert_evo_to_np(traj1_evo, traj1.shape)
    traj2_aligned = convert_evo_to_np(traj2_evo, traj2.shape)

    metric = evaluate_ape(traj1_aligned, traj2_aligned)
    return(metric.get_all_statistics())


In [ ]:
## Comparison between RH and robot end-eff
# (We'll want comparisons between the person's different demos, and the end-eff and the lh, also)

# robot_name='j2s6s300'
robot_name='Reachy'

followup = False

if followup:
    PID_max = 9
    gesture_max = 6
else: 
    PID_max = 16
    gesture_max = 15

for singlePIDval in range(1,PID_max+1):
    for gesture_num in range(1,gesture_max+1):
        end_eff, camera, rh, lh, joint = load_npzs(robot_name, PID, followup, gesture_num)
        #TODO Jennifer: ***ERROR*** total_joint is all zeros for Jaco right now!!! temp placeholder until we get JointMotion to save properly for the Jaco

        demo_max=5
        end_eff, camera, rh, lh, joints = segment_by_demo(total_end_eff, total_camera, total_rh, total_lh, total_joint, demo_max)
        # for i in range(0,5):
        #     plot_raw_data_subsampled(5, end_eff[i], camera[i], rh[i], lh[i], joint[i])

        for demo_num in range(0,5):
            end_eff_traj = get_evo_trajectory(end_eff[i])
            right_hand_traj = get_evo_trajectory(rh[i])   

            end_eff_traj, right_hand_traj = evo_sync(end_eff_traj, right_hand_traj)

            metric = evaluate_ape(end_eff_traj, right_hand_traj)
            print(metric.get_all_statistics()) # Not aligned statistics
            
            right_hand_traj = align(right_hand_traj, end_eff_traj)

            end_eff_aligned = convert_evo_to_np(end_eff_traj, end_eff[i].shape)
            # print(end_eff_aligned.shape)
            
            rh_aligned = convert_evo_to_np(right_hand_traj, rh[i].shape)
            # print(rh_aligned.shape)

            metric = evaluate_ape(end_eff_traj, right_hand_traj)
            metric.get_all_statistics() #Aligned statistics

In [ ]:
## Comparison between RH demos
# (We'll want comparisons between the person's different demos (both RH and LH, depending), and the end-eff and the hands, also)

# robot_name='j2s6s300'
robot_name='Reachy'

followup = False
demo_max = 5

if followup:
    PID_max = 9
    gesture_max = 6
else: 
    PID_max = 16
    gesture_max = 15


for singlePIDval in range(1,PID_max+1):
# for singlePIDval in range(1,2):    
    for gesture_num in range(1,gesture_max+1):
    # for gesture_num in range(1,2):
        end_eff, camera, rh, lh, joint = load_npzs(robot_name, PID, followup, gesture_num)
        #TODO Jennifer: ***ERROR*** total_joint is all zeros for Jaco right now!!! temp placeholder until we get JointMotion to save properly for the Jaco

        end_eff, camera, rh, lh, joints = segment_by_demo(total_end_eff, total_camera, total_rh, total_lh, total_joint, demo_max)
        # for i in range(0,5):
        #     plot_raw_data_subsampled(5, end_eff[i], camera[i], rh[i], lh[i], joint[i])

        demo_similarity = np.zeros([4,4,3])

        print("PID "+str(singlePIDval)+", gesture "+str(gesture_num))
        for demo_num1 in range(0,demo_max-1):
            for demo_num2 in range(demo_num1+1,demo_max):
                # print("PID "+str(singlePIDval)+", gesture "+str(gesture_num)+", demos "+str(demo_num1)+","+str(demo_num2))
                metrics1 = get_evo_metrics(rh[demo_num1],rh[demo_num2])
                # metrics2 = get_aligned_evo_metrics(rh[demo_num1],rh[demo_num2])
                # print(metrics1)
                # print(metrics1,metrics2)

                demo_similarity[demo_num1][demo_num2-1][0] = metrics1['rmse']
                demo_similarity[demo_num1][demo_num2-1][1] = metrics1['mean']
                demo_similarity[demo_num1][demo_num2-1][2] = metrics1['std']

        print(demo_similarity)

In [ ]:
metrics1['rmse']

In [ ]:
demo_similarity